# Projeto Data Mining II - Colorectal Cancer

In [1]:
# Import necessary libraries
import pandas as pd


In [2]:
# This script downloads a CSV file from a given URL and loads it into a pandas DataFrame.
url = "https://raw.githubusercontent.com/rgerc8/dm2-class-cancer/9bc81e82c839d3a4b14415d5bbd448452adad0f8/DM2%20Project%20Datasets/patient_train_data.csv"
df = pd.read_csv(url)

In [3]:
df.tail()

,ID,Alcohol Consumption,Cancer Stage,Country,Date of Birth,Diabetes,Diabetes History,Diet Risk,Early Detection,Family History,...,Non Smoker,Obesity BMI,Physical Activity,Screening History,Smoking History,Transfusion History,Treatment Type,Tumor Size (mm),Urban or Rural,Survival Prediction
75030,75031,Yes,Localized,Italy,06-07-1972,No,No,High,No,No,...,Yes,Obese,Moderate,Regular,No,-,Chemotherapy,42.0,Urban,No
75031,75032,Yes,Localized,USA,28-11-1954,No,No,Low,Yes,No,...,Yes,Overweight,Moderate,Regular,No,-,Chemotherapy,38.0,Urban,No
75032,75033,Yes,Metastatic,South Korea,27-04-1963,No,No,Moderate,Yes,Yes,...,Yes,Obese,Low,Regular,No,-,Surgery,60.0,NaN,Yes
75033,75034,No,Localized,South Africa,22-12-1951,No,No,High,Yes,No,...,Yes,Obese,Low,Never,No,-,Surgery,50.0,Rural,Yes
75034,75035,Yes,Regional,France,23-07-1948,No,No,Moderate,No,Yes,...,Yes,Overweight,High,Irregular,No,-,Combination,41.0,Urban,No


In [5]:
df.describe(include = ['O'])

,Alcohol Consumption,Cancer Stage,Country,Date of Birth,Diabetes,Diabetes History,Diet Risk,Early Detection,Family History,Gender,...,Insurance Status,Non Smoker,Obesity BMI,Physical Activity,Screening History,Smoking History,Transfusion History,Treatment Type,Urban or Rural,Survival Prediction
count,74983,75022,74985,74975,75021,75035,74987,75005,74989,74956,...,74974,75035,75011,74972,74982,74974,75035,74972,74984,75035
unique,2,3,16,11062,2,2,3,2,2,3,...,2,2,3,3,3,2,1,4,4,2
top,No,Regional,USA,09-06-1949,No,No,Moderate,Yes,No,M,...,Insured,Yes,Overweight,Moderate,Regular,No,-,Surgery,Urban,Yes
freq,37510,29882,11627,19,60078,75025,37630,44883,52458,44999,...,52776,44877,30035,29898,30152,44836,75035,29882,49834,45069


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75035 entries, 0 to 75034
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          75035 non-null  int64  
 1   Alcohol Consumption         74983 non-null  object 
 2   Cancer Stage                75022 non-null  object 
 3   Country                     74985 non-null  object 
 4   Date of Birth               74975 non-null  object 
 5   Diabetes                    75021 non-null  object 
 6   Diabetes History            75035 non-null  object 
 7   Diet Risk                   74987 non-null  object 
 8   Early Detection             75005 non-null  object 
 9   Family History              74989 non-null  object 
 10  Gender                      74956 non-null  object 
 11  Genetic Mutation            75005 non-null  object 
 12  Healthcare Access           74972 non-null  object 
 13  Healthcare Costs            749

In [4]:
df.head(5)

,ID,Alcohol Consumption,Cancer Stage,Country,Date of Birth,Diabetes,Diabetes History,Diet Risk,Early Detection,Family History,...,Non Smoker,Obesity BMI,Physical Activity,Screening History,Smoking History,Transfusion History,Treatment Type,Tumor Size (mm),Urban or Rural,Survival Prediction
0,1,No,Localized,UK,29-01-1966,No,No,Moderate,No,No,...,Yes,Overweight,Low,Regular,No,-,Chemotherapy,33.0,Urban,Yes
1,2,No,Regional,Japan,21-12-1958,No,No,Low,No,No,...,No,Normal,Low,Irregular,Yes,-,Chemotherapy,17.0,Urban,No
2,3,No,Localized,France,16-06-1959,No,No,Low,Yes,No,...,No,Normal,Moderate,Never,Yes,-,Surgery,34.0,Urban,Yes
3,4,Yes,Localized,Japan,18-01-1956,Yes,No,Moderate,No,No,...,Yes,Obese,Low,Regular,No,-,Radiotherapy,71.0,Urban,No
4,5,No,Regional,USA,04-05-1975,No,No,Moderate,Yes,Yes,...,Yes,Overweight,Low,Regular,No,-,Radiotherapy,48.0,Rural,Yes
